In [ ]:
function get_ab_phase(x0,y0,x1,y1,x2,y2,x3,y3,widd,V00,A,step,UUS_0)
    U_imp = []
    for x_t in x0:step:x1
        imps = [ impurity(V00, [x_t,y0], widd)]
        H1 = Himp(imps, H_Kin)
        EE, UU = eigen(dense(H1).data)
        EE, UUS = eigenstates(dense(H1))
        push!(U_imp,UUS)
        A *= (UUS_0[1]'*UUS[1]) / abs(UUS_0[1]'*UUS[1])
        UUS_0 = copy(UUS)
    end
    for y_t in y1:step:y2
        imps = [ impurity(V00, [x1,y_t], widd)]
        H1 = Himp(imps, H_Kin)
        EE, UU = eigen(dense(H1).data)
        EE, UUS = eigenstates(dense(H1))
        push!(U_imp,UUS)
        A *= (UUS_0[1]'*UUS[1]) / abs(UUS_0[1]'*UUS[1])
        UUS_0 = copy(UUS)
    end
    for x_t in x2:-step:x3
        imps = [ impurity(V00, [x_t,y2], widd)]
        H1 = Himp(imps, H_Kin)
        EE, UU = eigen(dense(H1).data)
        EE, UUS = eigenstates(dense(H1))
        push!(U_imp,UUS)
        A *= (UUS_0[1]'*UUS[1]) / abs(UUS_0[1]'*UUS[1])
        UUS_0 = copy(UUS)
    end
    for y_t in y3:-step:y0
        imps = [ impurity(V00, [x3,y_t], widd)]
        H1 = Himp(imps, H_Kin)
        EE, UU = eigen(dense(H1).data)
        EE, UUS = eigenstates(dense(H1))
        push!(U_imp,UUS)
        A *= (UUS_0[1]'*UUS[1]) / abs(UUS_0[1]'*UUS[1])
        UUS_0 = copy(UUS)
    end
    return angle(A)/pi, U_imp
end

In [ ]:
function get_ab_phase2(x0,y0,x1,y1,x2,y2,x3,y3,step,widd,V00,A,UUS_0,Nx, Ny, cut_off,sp_basis,co,H_Kin)
    U_imp = []
    for x_t in x0:step:x1
        imp = [ impurity(V00, [x_t,y0], widd)]
        VVV, basis_sub, P, Pt = Vpin2(imp[1], Nx, Ny, cut_off, sp_basis, co)
        H1 = H_Kin + VVV
        Total_H_sp = (H1' + H1)/2;
        EE, UUS = eigenstates(dense(Total_H_sp))
        push!(U_imp,UUS)
        A *= (UUS_0[1]'*UUS[1]) / abs(UUS_0[1]'*UUS[1])
        UUS_0 = copy(UUS)
    end
    for y_t in y1:step:y2
        imp = [ impurity(V00, [x1,y_t], widd)]
        VVV, basis_sub, P, Pt = Vpin2(imp[1], Nx, Ny, cut_off, sp_basis, co)
        H1 = H_Kin + VVV
        Total_H_sp = (H1' + H1)/2;
        EE, UUS = eigenstates(dense(Total_H_sp))
        push!(U_imp,UUS)
        A *= (UUS_0[1]'*UUS[1]) / abs(UUS_0[1]'*UUS[1])
        UUS_0 = copy(UUS)
    end
    for x_t in x2:-step:x3
        imp = [ impurity(V00, [x_t,y2], widd)]
        VVV, basis_sub, P, Pt = Vpin2(imp[1], Nx, Ny, cut_off, sp_basis, co)
        H1 = H_Kin + VVV
        Total_H_sp = (H1' + H1)/2;
        EE, UUS = eigenstates(dense(Total_H_sp))
        push!(U_imp,UUS)
        A *= (UUS_0[1]'*UUS[1]) / abs(UUS_0[1]'*UUS[1])
        UUS_0 = copy(UUS)
    end
    for y_t in y3:-step:y0
        imp = [ impurity(V00, [x3,y_t], widd)]
        VVV, basis_sub, P, Pt = Vpin2(imp[1], Nx, Ny, cut_off, sp_basis, co)
        H1 = H_Kin + VVV
        Total_H_sp = (H1' + H1)/2;
        EE, UUS = eigenstates(dense(Total_H_sp))
        push!(U_imp,UUS)
        A *= (UUS_0[1]'*UUS[1]) / abs(UUS_0[1]'*UUS[1])
        UUS_0 = copy(UUS)
    end
    return angle(A)/pi, U_imp
end

In [ ]:
function get_ab_phase_proj(x0,y0,x1,y1,x2,y2,x3,y3,widd,V00,A,step,UUS_0M,cut_off)
    U_imp = []
    for x_t in x0:step:x1
        imps = [ impurity(V00, [x_t,y0], widd)]
        H1 = Himp_proj(imps, H_Kin, cut_off, UUS_0)
        EE, UU = eigen(dense(H1).data)
        EE, UUS = eigenstates(dense(H1))
        push!(U_imp,UUS)
        A *= (UUS_0M[1]'*UUS[1]) / abs(UUS_0M[1]'*UUS[1])
        UUS_0M = copy(UUS)
    end
    for y_t in y1:step:y2
        imps = [ impurity(V00, [x1,y_t], widd)]
        H1 = Himp_proj(imps, H_Kin, cut_off, UUS_0)
        EE, UU = eigen(dense(H1).data)
        EE, UUS = eigenstates(dense(H1))
        push!(U_imp,UUS)
        A *= (UUS_0M[1]'*UUS[1]) / abs(UUS_0M[1]'*UUS[1])
        UUS_0M = copy(UUS)
    end
    for x_t in x2:-step:x3
        imps = [ impurity(V00, [x_t,y2], widd)]
        H1 = Himp_proj(imps, H_Kin, cut_off, UUS_0)
        EE, UU = eigen(dense(H1).data)
        EE, UUS = eigenstates(dense(H1))
        push!(U_imp,UUS)
        A *= (UUS_0M[1]'*UUS[1]) / abs(UUS_0M[1]'*UUS[1])
        UUS_0M = copy(UUS)
    end
    for y_t in y3:-step:y0
        imps = [ impurity(V00, [x3,y_t], widd)]
        H1 = Himp_proj(imps, H_Kin, cut_off, UUS_0)
        EE, UU = eigen(dense(H1).data)
        EE, UUS = eigenstates(dense(H1))
        push!(U_imp,UUS)
        A *= (UUS_0M[1]'*UUS[1]) / abs(UUS_0M[1]'*UUS[1])
        UUS_0M = copy(UUS)
    end
    return angle(A)/pi, U_imp
end

In [ ]:
function Himp_proj(imps, H_Kin, cut_off, UUS_0)
    VimpM = Vimp2(imps, sites, site_number, site_coord, LL);
    Vimp_op = get_sp_op(basis, VimpM)
    Total_Ham = H_Kin + Vimp_op # H_Kin ve Vimp_op ayrı ayrı proj yapmadım!!!!! ###
    substates = 1:cut_off
    basis_tilde = SubspaceBasis(basis, UUS_0[substates])
    basis_tilde2 = basis_tilde⊗basis_tilde
    states_mb_tilde = bosonstates(basis_tilde, 1)
    basis_mb_tilde  = ManyBodyBasis(basis_tilde, states_mb_tilde)
    P1 = projector(basis, basis_tilde)
    P1_dag = dagger(P1)
    P1M = P1.data
    P1_dagM = P1_dag.data
    P2M = (P1⊗P1).data
    P2_dagM = (P1_dag⊗P1_dag).data;
    H1M = Total_Ham.data
    H1_tildeM = P1_dagM * H1M * P1M
    H1_tilde = Operator(basis_tilde, H1_tildeM);
    H_NI_tilde_mb = manybodyoperator(basis_mb_tilde, H1_tilde);
    H_tilde_mb = (H_NI_tilde_mb + H_NI_tilde_mb')/2;
    return H_tilde_mb
end